In [1]:
import pandas as pd

data = pd.read_csv("raw_data/replies.csv")
data.head()

,_id,type,error,status,message,channel,createdat,userid,linkedinlimited,date,...,stepid,identityid,subtype,requeued,reply,duration,templateid,actionid,deleted,success
0,628e7357a0020c26468eeacc,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653502807905,607db83b997db13d4004d74b,False,NaN,...,909ff690-bc1f-439c-8a0c-817f9743d335,615dbcf23e99504c4faa2960,909ff690-bc1f-439c-8a0c-817f9743d335,False,Not something we need. Maybe in 18 months. Tha...,NaN,62798b6f9736365dce4c852e,NaN,False,True
1,62925729dd8678265fb5e7a7,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653757737452,60409f62d358a074403702b1,False,NaN,...,37c5a2a2-f62a-4326-ac6a-603c83756fbe,62165e8e4ed54d04c1acadad,37c5a2a2-f62a-4326-ac6a-603c83756fbe,False,"Bonjour Yohan, Merci pour votre message. Malh...",NaN,62165f64939d8a2c75db35c2,NaN,False,True
2,62925729dd8678265fb5e7a8,LINKEDIN_HAS_REPLY,False,SUCCESS,Reply: true - Connect,LINKEDIN,1653757737452,60409f62d358a074403702b1,False,NaN,...,37c5a2a2-f62a-4326-ac6a-603c83756fbe,62165e8e4ed54d04c1acadad,37c5a2a2-f62a-4326-ac6a-603c83756fbe,False,"Bonjour Yohan, Merci pour votre message. Malh...",NaN,62165f64939d8a2c75db35c2,NaN,False,True
3,62925559ebad282654c1abd9,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653757273534,628fc9ccebad282654bf7337,False,NaN,...,59ae5c2d-961a-4eac-abe3-885652f7630c,628fc9d105f9d3263f7a05d6,59ae5c2d-961a-4eac-abe3-885652f7630c,False,Hi Mayank,NaN,628fce89ebad282654bf77f0,NaN,False,True
4,62925559ebad282654c1abda,LINKEDIN_HAS_REPLY,False,SUCCESS,Reply: true - Connect,LINKEDIN,1653757273534,628fc9ccebad282654bf7337,False,NaN,...,59ae5c2d-961a-4eac-abe3-885652f7630c,628fc9d105f9d3263f7a05d6,59ae5c2d-961a-4eac-abe3-885652f7630c,False,Hi Mayank,NaN,628fce89ebad282654bf77f0,NaN,False,True


In [2]:
data.message

0          Replied to DM: true
1          Replied to DM: true
2        Reply: true - Connect
3          Replied to DM: true
4        Reply: true - Connect
                 ...          
95269    Reply: true - Connect
95270             Reply: true 
95271    Reply: true - Connect
95272      Replied to DM: true
95273    Reply: true - Connect
Name: message, Length: 95274, dtype: object

In [3]:
#to activate when target available

#from sklearn import preprocessing

#le = preprocessing.LabelEncoder()
#data["target_encoded"] =  le.fit_transform(data.target)

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/Users/pierre/.pyenv/versions/3.8.13/envs/lgm-le-wagon/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [7]:
from transformers import pipeline

multilang_classifier = pipeline("sentiment-analysis", 
                                model="nlptown/bert-base-multilingual-uncased-sentiment")

2022-06-02 18:22:16.037389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
multilang_classifier(data.reply[8], return_all_scores=True)

[[{'label': '1 star', 'score': 0.10444572567939758},
  {'label': '2 stars', 'score': 0.09467915445566177},
  {'label': '3 stars', 'score': 0.2151179015636444},
  {'label': '4 stars', 'score': 0.277179479598999},
  {'label': '5 stars', 'score': 0.3085777461528778}]]

In [9]:
multilang_classifier

In [10]:
replies = data.drop_duplicates(["leadid", "campaignid", "reply"])

In [11]:
to_save = replies['reply']

In [12]:
to_save.to_csv('replies-target2.csv',sep=';',encoding='UTF-8',header=False)

In [13]:
labelized_replies = pd.read_csv('replies-label.csv')
labelized_replies.shape

(1341, 4)

In [14]:
cleaned_labelized_replies = labelized_replies.dropna(axis=0)
cleaned_labelized_replies.shape

(1335, 4)

In [15]:
label_to_merge = cleaned_labelized_replies.drop(columns=['id','text'])

In [16]:
merge = replies.merge(label_to_merge,how='left',on='_id')

In [17]:
merge

,_id,type,error,status,message,channel,createdat,userid,linkedinlimited,date,...,identityid,subtype,requeued,reply,duration,templateid,actionid,deleted,success,label
0,628e7357a0020c26468eeacc,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653502807905,607db83b997db13d4004d74b,False,NaN,...,615dbcf23e99504c4faa2960,909ff690-bc1f-439c-8a0c-817f9743d335,False,Not something we need. Maybe in 18 months. Tha...,NaN,62798b6f9736365dce4c852e,NaN,False,True,neutral
1,62925729dd8678265fb5e7a7,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653757737452,60409f62d358a074403702b1,False,NaN,...,62165e8e4ed54d04c1acadad,37c5a2a2-f62a-4326-ac6a-603c83756fbe,False,"Bonjour Yohan, Merci pour votre message. Malh...",NaN,62165f64939d8a2c75db35c2,NaN,False,True,negative
2,62925559ebad282654c1abd9,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653757273534,628fc9ccebad282654bf7337,False,NaN,...,628fc9d105f9d3263f7a05d6,59ae5c2d-961a-4eac-abe3-885652f7630c,False,Hi Mayank,NaN,628fce89ebad282654bf77f0,NaN,False,True,NaN
3,6292b849ebad282654c1db70,LINKEDIN_HAS_REPLY,False,SUCCESS,Reply: true - Connect,LINKEDIN,1653782601451,6215ee75da45f704b6176b07,False,NaN,...,6215ee79f5933c04a8d19425,688ba81f-818d-4c4f-956a-2c50173c71de,False,"Bonjour Antoine, Au plaisir de lire vos posts,...",NaN,NaN,NaN,False,True,neutral
4,628e7376ebad282654be48cf,LINKEDIN_HAS_REPLY,False,SUCCESS,Reply: true - Connect,LINKEDIN,1653502838423,60409f62d358a074403702b1,False,NaN,...,60487f19c2044933f31844ed,c6530506-9db7-4acb-bb10-3110428a91ee,False,Disons q4 :),NaN,624af6dd9e7b445d082f6d62,NaN,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54447,6294f136a0020c2646953295,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653928246658,5d39b1787b53a58d195baaaf,False,NaN,...,61e8511683e1295b27b8d001,8d6598d8-7ed9-4bbd-bb0b-074dc60a042e,False,Merci,NaN,62714f6f13ee8d0bf05c44d9,NaN,False,True,NaN
54448,6294efe2ebad282654c494e4,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653927906550,6245a803009dbc7d46736429,False,NaN,...,6245a80880149a7d40a28d93,NaN,False,"Dear sender, I'll be away until next week. If ...",NaN,624aa7bb28eb2b5d0f172983,NaN,False,True,NaN
54449,6294f09205f9d3263f7f290a,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653928082370,623991ecd86d7946dd729f1a,False,NaN,...,624da67bb565b573bd2ae779,2fadd68a-d0de-4f16-a482-fd51557dfb71,False,"Bonjour David, Navré pour ma réponse tardive....",NaN,6287b45a4ac76d4728bc15e0,NaN,False,True,NaN
54450,6294f06733904114ef2ee051,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653928039533,620b8a655d37c17677bac5f2,False,NaN,...,620b8a6a56a110765e2076fa,NaN,False,Bonjour En déplacement du lundi 30 mai 2022 a...,NaN,625429fab565b573bd33447d,NaN,False,True,NaN


In [18]:
df_replies=pd.DataFrame(merge['reply']).dropna()

In [19]:
df_replies[df_replies['reply'].str.contains("absen")]

,reply
77,"Bonjour, Actuellement absente, avec un accès ..."
99,"Bonjour, Je suis actuellement absente et sera..."
112,"Bonjour, je suis absent du mercredi 25 mai au ..."
147,"Bonjour, je suis absente du bureau du 23/05/20..."
204,"Hello, I'm out of the office today 14th of fe..."
...,...
54311,"Bonjour, Merci pour votre message, actuelleme..."
54317,"Hi, I am out of the office until Mon 6th June..."
54352,"Bonjour, Actuellement absente, vous pouvez co..."
54436,"Bonjour, Je suis absent jusqu'au 30/05. Je pr..."


In [20]:
df_replies[df_replies['reply'].str.contains("office")]

,reply
15,"Hi, i'm out of the office until the 30th of m..."
21,"First of all, thank you for your good behavior..."
22,Not my area... contact our corporate office in...
103,"Hi, Thank you for your email. I am out of the ..."
105,"Dear sender, I am currently out of office and ..."
...,...
54399,"Hello, Thank you for your email. I will be ou..."
54422,"Dear Sender, Thank you for your e-mail. I am c..."
54429,"Bonjour, Absente du 25/05/2022 au 31/05/2022 ..."
54439,"Hello, I am out of the office, back on May 31s..."


In [21]:
terms = ['congé', 'absen', 'restreint', 'vacances', 'ooo']
merge[merge['reply'].str.contains("|".join(terms),na=False)]

,_id,type,error,status,message,channel,createdat,userid,linkedinlimited,date,...,identityid,subtype,requeued,reply,duration,templateid,actionid,deleted,success,label
39,628e8cab05f9d3263f78f3e7,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653509291932,603fa0f2edb37d744754ab3c,False,NaN,...,60cb1dbc61147e7726e22e75,419f63a7-66ab-4bee-88e2-afd52fb23052,False,"Bonjour Clémence, Je suis en effet à la recher...",NaN,625ed173c754f419b71c100a,NaN,False,True,NaN
54,628e9874dd8678265fb2acd7,LINKEDIN_ON_REPLY,False,SUCCESS,Replied to DM: true,LINKEDIN,1653512308261,6155bcc031fc674c480ff9ed,False,NaN,...,6221dd5bb11c700a50cf636d,df96f2cb-aa14-454f-a172-95d5abba6b73,False,Bonjour Désolé je suis actuellement en congés...,NaN,627a8fe99b80b427c1e19551,NaN,False,True,NaN
77,628eb3b305f9d3263f7912b6,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653519283400,6215ee75da45f704b6176b07,False,NaN,...,6215ee79f5933c04a8d19425,NaN,False,"Bonjour, Actuellement absente, avec un accès ...",NaN,627cdd0400b7b35e91f6c0be,NaN,False,True,NaN
99,628ee29005f9d3263f792d3c,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653531280699,615a9d4731fc674c48122ee5,False,NaN,...,615a9d4c0ee40f4c5698ff23,NaN,False,"Bonjour, Je suis actuellement absente et sera...",NaN,628707075363ea470f9e7b25,NaN,False,True,NaN
112,628f1171dd8678265fb2ef75,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653543281288,615a9d4731fc674c48122ee5,False,NaN,...,615a9d4c0ee40f4c5698ff23,NaN,False,"Bonjour, je suis absent du mercredi 25 mai au ...",NaN,62616ec541e6766974cf647f,NaN,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54352,6294de4f05f9d3263f7eed85,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653923407362,6038a95c53aa0774591f3d12,False,NaN,...,603f43a91cb174746468a43f,NaN,False,"Bonjour, Actuellement absente, vous pouvez co...",NaN,628209d7bbd61a7051a8851d,NaN,False,True,NaN
54398,6294e553ebad282654c4742b,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653925203266,6127a29c6342312afe04204c,False,NaN,...,6127b8d6cc73bc2ae95e72e0,NaN,False,"Salut Thomas, Très bien je te remercie, j’esp...",NaN,628e2853dd8678265fb1bee9,NaN,False,True,NaN
54436,6294ec6105f9d3263f7f1d3e,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653927009018,62728e1cc8cab40be1ea5b34,False,NaN,...,62728e2213ee8d0bf05e5366,NaN,False,"Bonjour, Je suis absent jusqu'au 30/05. Je pr...",NaN,6273dad165b47b0bdac458ab,NaN,False,True,NaN
54444,6294eec1ebad282654c4925c,GOOGLE_REPLY,False,SUCCESS,Reply: true,GOOGLE,1653927617925,61efc10d39f30a134d7bf111,False,NaN,...,62027a0e23f20665eb41b086,NaN,False,"Bonjour, actuellement en congé maternité, je ...",NaN,62948228ebad282654c300be,NaN,False,True,NaN


In [22]:
ooo_df = merge[['_id','reply','label']].copy()
ooo_df

,_id,reply,label
0,628e7357a0020c26468eeacc,Not something we need. Maybe in 18 months. Tha...,neutral
1,62925729dd8678265fb5e7a7,"Bonjour Yohan, Merci pour votre message. Malh...",negative
2,62925559ebad282654c1abd9,Hi Mayank,NaN
3,6292b849ebad282654c1db70,"Bonjour Antoine, Au plaisir de lire vos posts,...",neutral
4,628e7376ebad282654be48cf,Disons q4 :),NaN
...,...,...,...
54447,6294f136a0020c2646953295,Merci,NaN
54448,6294efe2ebad282654c494e4,"Dear sender, I'll be away until next week. If ...",NaN
54449,6294f09205f9d3263f7f290a,"Bonjour David, Navré pour ma réponse tardive....",NaN
54450,6294f06733904114ef2ee051,Bonjour En déplacement du lundi 30 mai 2022 a...,NaN


In [23]:
labels = ['positive', 'negative', 'neutral']
terms = ['congé', 'absen', 'restreint', 'vacances', 'ooo']
ooo_df[ooo_df['reply'].str.contains("|".join(terms),na=False)]

,_id,reply,label
39,628e8cab05f9d3263f78f3e7,"Bonjour Clémence, Je suis en effet à la recher...",NaN
54,628e9874dd8678265fb2acd7,Bonjour Désolé je suis actuellement en congés...,NaN
77,628eb3b305f9d3263f7912b6,"Bonjour, Actuellement absente, avec un accès ...",NaN
99,628ee29005f9d3263f792d3c,"Bonjour, Je suis actuellement absente et sera...",NaN
112,628f1171dd8678265fb2ef75,"Bonjour, je suis absent du mercredi 25 mai au ...",NaN
...,...,...,...
54352,6294de4f05f9d3263f7eed85,"Bonjour, Actuellement absente, vous pouvez co...",NaN
54398,6294e553ebad282654c4742b,"Salut Thomas, Très bien je te remercie, j’esp...",NaN
54436,6294ec6105f9d3263f7f1d3e,"Bonjour, Je suis absent jusqu'au 30/05. Je pr...",NaN
54444,6294eec1ebad282654c4925c,"Bonjour, actuellement en congé maternité, je ...",NaN


In [24]:
#ooo_df[ooo_df['label'].str.contains("|".join(labels),na=False)]

In [25]:
#ooo_df['ooo'] = ooo_df['label'].map(lambda x: is_not_out_office(x))

In [26]:
ooo_df

,_id,reply,label
0,628e7357a0020c26468eeacc,Not something we need. Maybe in 18 months. Tha...,neutral
1,62925729dd8678265fb5e7a7,"Bonjour Yohan, Merci pour votre message. Malh...",negative
2,62925559ebad282654c1abd9,Hi Mayank,NaN
3,6292b849ebad282654c1db70,"Bonjour Antoine, Au plaisir de lire vos posts,...",neutral
4,628e7376ebad282654be48cf,Disons q4 :),NaN
...,...,...,...
54447,6294f136a0020c2646953295,Merci,NaN
54448,6294efe2ebad282654c494e4,"Dear sender, I'll be away until next week. If ...",NaN
54449,6294f09205f9d3263f7f290a,"Bonjour David, Navré pour ma réponse tardive....",NaN
54450,6294f06733904114ef2ee051,Bonjour En déplacement du lundi 30 mai 2022 a...,NaN


In [27]:
def is_out_of_office_max(x,y):
    string = str(y)
    substring = 'absen'
    if x == 'negative':
        return 0
    elif x == 'neutral':
        return 0
    elif x == 'positive':
        return 0
    elif substring in string:
        return 1
    else:
        return 'unknow'
    


In [28]:
ooo_df['ooo'] = ooo_df.apply(lambda x: is_out_of_office_max(x['label'],x['reply']), axis=1)

In [29]:
ooo_df

,_id,reply,label,ooo
0,628e7357a0020c26468eeacc,Not something we need. Maybe in 18 months. Tha...,neutral,0
1,62925729dd8678265fb5e7a7,"Bonjour Yohan, Merci pour votre message. Malh...",negative,0
2,62925559ebad282654c1abd9,Hi Mayank,NaN,unknow
3,6292b849ebad282654c1db70,"Bonjour Antoine, Au plaisir de lire vos posts,...",neutral,0
4,628e7376ebad282654be48cf,Disons q4 :),NaN,unknow
...,...,...,...,...
54447,6294f136a0020c2646953295,Merci,NaN,unknow
54448,6294efe2ebad282654c494e4,"Dear sender, I'll be away until next week. If ...",NaN,unknow
54449,6294f09205f9d3263f7f290a,"Bonjour David, Navré pour ma réponse tardive....",NaN,unknow
54450,6294f06733904114ef2ee051,Bonjour En déplacement du lundi 30 mai 2022 a...,NaN,unknow


In [30]:
model_df = ooo_df[ooo_df['ooo'] != 'unknow']

In [31]:
X = model_df['reply'].values

In [32]:
y = model_df['ooo'].values

In [33]:
y = y.astype('int64')

In [34]:
X

array(['Not something we need. Maybe in 18 months. Thanks! ',
       'Bonjour Yohan,  Merci pour votre message. Malheureusement je viens de commencer un nouveau contrat dans un cabinet de conseil. Donc pas de changement de prévu pour tout de suite 🙂  Bonne journée ! Yajing',
       'Bonjour Antoine, Au plaisir de lire vos posts, je vous souhaite une très belle fin de journée. Damien J EVEQUOZ  ',
       ...,
       'Bonjour,  Actuellement absente, vous pouvez contacter mathieu.deschamps@boursorama.fr en cas de besoin.  Cordialement Aude  ',
       "Bonjour,  Je suis absent jusqu'au 30/05. Je prendrai connaissance de votre message à mon retour le 31/05. En cas d'urgence, merci de contacter Célia Rierny celia.rierny@ocp.fr.  Cordialement  Aymeric   ",
       'Bonjour,  Je suis absente et serai de retour mercredi 1er juin.  Merci, Yseult ----------------  Hello,  I am out of office and will be back on Wednesday June 1st.  Thanks, Yseult   ________________________________   This e-mail may

In [35]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [38]:
y_train

array([1, 0, 1, ..., 1, 1, 1])

In [39]:
X_train_word = [text_to_word_sequence(_,lower=False) for _ in X_train]
X_test_word = [text_to_word_sequence(_,lower=False) for _ in X_test]

In [40]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train_word, vector_size=60, min_count=10, window=10)

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [42]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

In [43]:
# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [44]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

In [45]:
from sklearn.metrics import accuracy_score

unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)

y_pred = 0 if counts[0] > counts[1] else 1


Number of labels in train set {0: 607, 1: 1888}


In [46]:
print('Baseline accuracy: ', accuracy_score(y_test, [y_pred]*len(y_test)))

Baseline accuracy:  0.7598130841121495


In [47]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

model = init_model()

In [48]:
#import tensorflow as tf

In [49]:
y_test

array([0, 1, 1, ..., 1, 1, 1])

In [50]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
55/55 [==============================] - 14s 139ms/step - loss: 0.5691 - accuracy: 0.7474 - val_loss: 0.5186 - val_accuracy: 0.7704
Epoch 2/100
55/55 [==============================] - 6s 116ms/step - loss: 0.5384 - accuracy: 0.7514 - val_loss: 0.5221 - val_accuracy: 0.7704
Epoch 3/100
55/55 [==============================] - 7s 124ms/step - loss: 0.5286 - accuracy: 0.7514 - val_loss: 0.5045 - val_accuracy: 0.7824
Epoch 4/100
55/55 [==============================] - 7s 129ms/step - loss: 0.5170 - accuracy: 0.7532 - val_loss: 0.4955 - val_accuracy: 0.7810
Epoch 5/100
55/55 [==============================] - 8s 142ms/step - loss: 0.5076 - accuracy: 0.7532 - val_loss: 0.4642 - val_accuracy: 0.7824
Epoch 6/100
55/55 [==============================] - 9s 155ms/step - loss: 0.5098 - accuracy: 0.7572 - val_loss: 0.4582 - val_accuracy: 0.7824
Epoch 7/100
55/55 [==============================] - 7s 133ms/step - loss: 0.4812 - accuracy: 0.7652 - val_loss: 0.4379 - val_accuracy: 0.794

In [51]:
res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 81.869%


In [52]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [54]:
word2vec_transfer = api.load("glove-wiki-gigaword-300")

[=================================================-] 99.7% 374.9/376.1MB downloaded


In [55]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [56]:
# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [60]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

model_transfer = init_model()

In [61]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

#model = init_model()

history = model_transfer.fit(X_train_pad_2, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
55/55 [==============================] - 25s 233ms/step - loss: 0.5598 - accuracy: 0.7491 - val_loss: 0.5153 - val_accuracy: 0.7717
Epoch 2/100
55/55 [==============================] - 15s 278ms/step - loss: 0.5315 - accuracy: 0.7583 - val_loss: 0.4966 - val_accuracy: 0.7797
Epoch 3/100
55/55 [==============================] - 11s 200ms/step - loss: 0.5062 - accuracy: 0.7738 - val_loss: 0.4894 - val_accuracy: 0.7797
Epoch 4/100
55/55 [==============================] - 10s 187ms/step - loss: 0.4855 - accuracy: 0.7904 - val_loss: 0.4278 - val_accuracy: 0.8224
Epoch 5/100
55/55 [==============================] - 11s 198ms/step - loss: 0.4538 - accuracy: 0.8007 - val_loss: 0.4168 - val_accuracy: 0.8331
Epoch 6/100
55/55 [==============================] - 12s 225ms/step - loss: 0.4231 - accuracy: 0.8173 - val_loss: 0.4388 - val_accuracy: 0.7877
Epoch 7/100
55/55 [==============================] - 11s 196ms/step - loss: 0.3952 - accuracy: 0.8247 - val_loss: 0.3554 - val_accuracy:

In [63]:
res_transfer = model_transfer.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res_transfer[1]*100:.3f}%')

The accuracy evaluated on the test set is of 91.121%
